# Here we run tests that show whether the compilation process is working or not.
# These tests should be run whenever the user changes the code, to ensure that there are no unexpected bugs.

In [1]:
# Core Python packages
import numpy as np

# Qiskit core components
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import RXXGate

# Custom modules for compilation
from fidelity_measures import circuit_fidelity
from circuit import compile


In [2]:
# Set up: choose the basis for decomposition, qubit number, circuit depth and Noise model

basis_gate = RXXGate(np.pi/2)  # basis gate
euler_basis = "XYX"  # Euler basis 

number_of_qubits = 2 # choose number of qubits
circuit_depth = 1 # choose depth of circuit

total_noise_model = None  # no noise model for tests

In [3]:
# Test 1: Decompose a tensor product gate

# Description: Create a two qubit gate which is the tensor product of two single qubit gates.
# i.e. it has no entangling capability. 

# Expected output:
# The algorithm should find that the optimal decomposition occcurs with ZERO instances of the two qubit basis gate.

# Create ideal quantum circuit (no decomposition, no noise)
qc_exact = QuantumCircuit(number_of_qubits, number_of_qubits)

# Create noisy quantum circuit (decomposition and noise)
qc_compiled = QuantumCircuit(number_of_qubits, number_of_qubits)

# define qubit numbers
m=0
n=1

# create target unitary out of pauli-z gates on each qubit
target_unitary = Operator(np.kron(np.array([[1, 0], [0, -1]], dtype=complex), 
                                  np.array([[1, 0], [0, -1]], dtype=complex)))

# # create target unitary out of random single qubit gates
# (sometimes this will have the highest fidelity with more than 0 basis gates
# which is due to numerical noise
# Only ever with 0 or 2 instances of the basis gate, I think when it chooses 2 it is essentially 
# undoing any entanglement created by the first instance of the basis gate, therefore giving itself more single qubit gates
# to approximate the target with in the decomposition)
# target_unitary = Operator(np.kron(random_unitary(2).data,
#                                   random_unitary(2).data)) 
                                  
# apply the target unitary to the ideal circuit
qc_exact.unitary(target_unitary, [m, n], label='target unitary')

# run compilation 
qc_compiled = compile(target_unitary, basis_gate, euler_basis, qc_compiled, m, n, noise_model=None)

# run both circuits, calculate fidelity
fid, counts_exact, counts_decomp = circuit_fidelity(number_of_qubits, qc_exact, qc_compiled, total_noise_model)

# test the fidelity
if fid >= 0.99:
    print(f'Fidelity = {fid:.10f} -- Compilation successful!')

Fidelity = 1.0000000000 -- Compilation successful!


In [4]:
# Test 2: Remove the noise model

# Description: Remove the noise model from the algorithm and run the decomposition.

# Expected output:
# The algorithm should choose the number of basis gates which gives the highest fidelity (which will depend on the target unitary being decomposed),
# regardless, the fidelity should be approximately 1. 


# Create ideal quantum circuit (no decomposition, no noise)
qc_exact = QuantumCircuit(number_of_qubits, number_of_qubits)

# Create noisy quantum circuit (decomposition and noise)
qc_compiled = QuantumCircuit(number_of_qubits, number_of_qubits)

# define qubit numbers
m=0
n=1

# create target unitary gate
target_unitary = Operator(random_unitary(4))

# apply the target unitary to the ideal circuit
qc_exact.unitary(target_unitary, [m, n], label='target unitary')

# run compilation 
qc_compiled = compile(target_unitary, basis_gate, euler_basis, qc_compiled, m, n, noise_model=None)

# run both circuits, calculate fidelity
fid, counts_exact, counts_decomp = circuit_fidelity(number_of_qubits, qc_exact, qc_compiled, total_noise_model)

# test the fidelity
if fid >= 0.99:
    print(f'Fidelity = {fid:.10f} -- Compilation successful!')


Fidelity = 0.9972680573 -- Compilation successful!
